# Сравнение 1NN и RandomForest на примере датасета digits
## Реализация 1NN

In [1]:
from sklearn.datasets import load_digits

X, y = load_digits(return_X_y = True)

In [2]:
from sklearn.model_selection import train_test_split

# формируем фиксированное разбиение на обучающую выборку и тест в соотношении 3:1
train_size = int(round(len(y)*0.75))

X_train = X[:train_size, :]
y_train = y[:train_size]
X_test  = X[train_size:, :]
y_test  = y[train_size:]

In [3]:
# test_distances - список расстояний i-го элемента тестовой выборки до каждого из классов
# минимальные расстояния каждого элемента до каждого из классов представлены словарем {класс: мин.расстояние}
test_distances = list()

# перебираем все элементы тестовой выборки
for i, test_el in enumerate(X_test):
    # добавляем элемент - словарь минимальных расстояний до каждого из классов
    test_distances.append(dict())
    # перебираем все элементы обучающей выборки, считаем расстояние от test_el до каждого из них
    for train_el, train_cl in zip(X_train, y_train):
        dst = 0
        #посчитаем расстояние от элемента test_el до train_el
        for test_el_i, train_el_i in zip(test_el, train_el):
            dst += (test_el_i - train_el_i)**2 # корень не извлекаем для экономии времени
        
        # если расстояние от test_el до train_el меньше, чем уже рассмотренные 
        # расстояния до элементов того же класса - добавляем расстояние и класс в словарь
        if train_cl not in test_distances[i] or test_distances[i][train_cl] > dst:
            test_distances[i][train_cl] = dst

In [4]:
# предсказываем класс на тестовой выборке: y_test_predict - предсказанные классы
y_test_predict = []
# перебираем все списки расстояний test_distances
for i in range(len(test_distances)):
    # для каждого элемента находим класс, расстояние до которого меньше, чем до всех остальных классов
    min_class, min_distance = test_distances[i].items()[0]
    for k, v in test_distances[i].items():
        if v < min_distance:
            min_class, min_distance = k, v
    # ближайший класс добавляем в предсказание
    y_test_predict.append(min_class)


In [5]:
# считаем долю ошибок
cnt = 0
for i in range(len(y_test_predict)):
    if y_test_predict[i] != y_test[i]:
        cnt += 1

print u"Доля ошибок:", cnt/float(len(y_test))

Доля ошибок: 0.0378619153675


Посмотрим, как на том же датасете отработает случайный лес

In [6]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1000, n_jobs = -1, verbose=1)
rf.fit(X_train, y_train)
y_test_predict_rf = rf.predict(X_test)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    1.7s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    0.9s finished


In [7]:
print u'Доля ошибок в лесу:', 1-rf.score(X_test, y_test)

Доля ошибок в лесу:

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.3s


 0.0668151447661


[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    1.6s finished


Видим, что простейший метод 1NN на данном датасете работает лучше, чем случайный лес на 1000 деревьев.
Вывод: не стоит пренебрегать простейшими методами